In [1]:
import house_data
import pandas as pd
import requests
from bs4 import BeautifulSoup

mcr_url = "https://www.zoopla.co.uk/for-sale/property/greater-manchester/?view_type=list&q=Greater%20Manchester&results_sort=newest_listings&search_source=home"
london_url = "https://www.zoopla.co.uk/for-sale/property/london/?view_type=list&q=London&results_sort=newest_listings&search_source=home"

#first_url = "https://www.zoopla.co.uk/for-sale/property/greater-manchester/hyde/?q=Hyde&results_sort=newest_listings&search_source=for-sale"
df = pd.DataFrame()

In [2]:

def get_data(url):
    df = pd.DataFrame()
    pages = 1
    numbers = []

    soup = BeautifulSoup(requests.get(url + "&pn=1").content)
    page_buttons = soup.findAll("a", {"class":"eaoxhri5 css-xtzp5a-ButtonLink-Button-StyledPaginationLink eaqu47p1"})
    for num in page_buttons:
        try:
            numbers.append(int(num.text))
        except:
            pass

    number_of_pages = max(numbers)

    error_count = 0
    for i in range(number_of_pages):
        try:
            soup = BeautifulSoup(requests.get(url + "&pn=" + str(i)).content)
            items = soup.find("div", {"class":"css-1anhqz4-ListingsContainer earci3d2"})

            index = 1

            for listing in items.select('div[data-testid*="search-result_listing"]'):
                try:

                    title = listing.find("h2", {"data-testid":"listing-title"}).text
                    address = listing.find("p", {"data-testid":"listing-description"}).text
                    price = listing.find("p", {"size":"6"}).text
                    station = listing.find("p", {"class":"css-nwapgq-Text eczcs4p0", "data-testid":"text"}).text
                    
                    home_info = listing.findAll("p", {"class":"css-r8a2xt-Text eczcs4p0", "data-testid":"text"})
                    
                    #print(len(home_info))
                    
                    beds = 0
                    baths = 0
                    receptions = 0

                    if (len(home_info) == 3):
                        beds = home_info[0].text
                        baths = home_info[1].text
                        receptions = home_info[2].text
                    elif (len(home_info) == 2):
                        beds = home_info[0].text
                        baths = home_info[1].text
                    else:
                        beds = home_info[0].text

                    new_data = {'title':title, 'address':address, 'price':price, 'station':station, 'beds':beds, 'baths':baths, 'receptions':receptions}
                    df = df.append(new_data,ignore_index=True)

                except:
                    error_count += 1
         
                #print(index)
                #print(f"{index} - {title}")
                #print(f"Address: {address}")
                #print(f"Price: {price}")
                #print(f"Station: {station}")


                #print(listing.find("div", {"data-testid":"listing-transport"}).contents)
            
                index +=1
        except:
            error_count += 25
            print(f"Invalid page {i}")
    

    print(f"Error count: {error_count}")
    return df

In [3]:
data_frames = []

In [4]:
data_frames.append(get_data(mcr_url))
data_frames.append(get_data(london_url))


Error count: 53
Error count: 101


In [5]:
data_frames

[                                   title  \
 0       2 bed end terrace house for sale   
 1     2 bed semi-detached house for sale   
 2          4 bed detached house for sale   
 3     3 bed semi-detached house for sale   
 4                    3 bed flat for sale   
 ...                                  ...   
 5992                 1 bed flat for sale   
 5993       2 bed terraced house for sale   
 5994             4 bed bungalow for sale   
 5995      1 bed barn conversion for sale   
 5996                 1 bed flat for sale   
 
                                                 address     price  \
 0                                   Old Road, Hyde SK14  £140,000   
 1             Woodland Avenue, Hindley Green, Wigan WN2  £135,000   
 2     Halfpenny Bridge Industrial Estate, Lincoln St...  £294,950   
 3                              Wilfred Road, Eccles M30  £250,000   
 4      Manchester City Centre Apartments, Manchester M2  £450,000   
 ...                                  

In [6]:
df = pd.concat(data_frames)

In [7]:
df

,title,address,price,station,beds,baths,receptions
0,2 bed end terrace house for sale,"Old Road, Hyde SK14","£140,000",0.1 miles Flowery Field,2,1,2
1,2 bed semi-detached house for sale,"Woodland Avenue, Hindley Green, Wigan WN2","£135,000",1.3 miles Hindley,2,1,1
2,4 bed detached house for sale,"Halfpenny Bridge Industrial Estate, Lincoln St...","£294,950",0.2 miles Castleton (Manchester),4,3,1
3,3 bed semi-detached house for sale,"Wilfred Road, Eccles M30","£250,000",1 miles Patricroft,3,1,2
4,3 bed flat for sale,"Manchester City Centre Apartments, Manchester M2","£450,000",0.4 miles Manchester Victoria,3,2,1
...,...,...,...,...,...,...,...
9894,3 bed property for sale,"Totteridge Lane, Whetstone, London N20","£850,000",0.1 miles Totteridge & Whetstone,3,2,2
9895,3 bed flat for sale,"Bladgon Road, New Malden KT3","£600,000",0.3 miles New Malden,3,2,1
9896,4 bed detached house for sale,"Woodlands Avenue, Coombe, New Malden KT3","£1,850,000",0.7 miles Norbiton,4,3,2
9897,4 bed terraced house for sale,"Batchelor Street, Islington, London N1","£1,600,000",0.2 miles Angel,4,3,2


In [ ]:
df.to_csv("databases/zooplai.csv")